In [ ]:
import numpy as np
import math 
import pandas as pd
import time
import plotly.graph_objects as go
import random

### Function

In [ ]:
#Stableswap function that outputs token supply of Y, given X, A factor, and D (the sum of the two tokens when in parity)
def sampleStable(x = 5,A = 10,D = 100, n = 2):
    if x<D:
        y = (-(A*4)*(x**2) + A*D*x*n**n - D*x + math.sqrt(((A*n**n)*(x**2) - A*D*x*n**n + D*x)**2 + 4*A*x*n**n*((D**(n+1))/n**n)))/(2*A*x*n**n)
        return y

#given the usdc side, and the price, output the bean and usdc in pool at price, and total assets (D) when in parity
def stableSwap(usdc,pri,A = 10):
    if A == 0:
        #special case for XYK
        return [usdc/pri, usdc, 2*math.sqrt(usdc*usdc/pri)]
    inital_D = usdc*2
    inital_bean = usdc
    #simple for loop that estimates conditions
    for i in np.linspace(0,1,1000):
        #TLDR: trying to find D: we start with the inital D, and increase it stepwise until we find the correct D, which then we can find the bean assoicated with it) 
        temp_D = inital_D/(1+i)
        amtOut = sampleStable(usdc,A,temp_D,2) - sampleStable(usdc + 1,A,temp_D,2)
        #since this is a guesstimate (an accurate one, but still), we simply compare them lmao
        if abs(pri - (1/amtOut)) < 0.00008 :
            newBean = sampleStable(usdc,10,temp_D,2)
            D = temp_D
            break
    return [newBean,usdc,D]

#given bean,usdc,D size, and A factor, output price 
def getPrice(Bean,usdc,D, amtIn = 1,A=10):
    amtOut =sampleStable(usdc,A,D,2) - sampleStable(usdc + amtIn,A,D,2)
    price = amtIn/amtOut
    return price

#given bean,usdc,D, gives the delta (mints needed to get back to peg)
def getDelta(bean,usdc,D):
    target = D/2
    delta = target - bean
    return delta 

#very rough calc, just takes the current mint, and finds the rough apr by exploiating it to a year 
def roughAPR(mints,stalk):
    beanPerStalk = mints/stalk
    APR = beanPerStalk*24*365

#this simulates a cycle of demand (there may be times where there is a lot of mints, and times where there is no mints)
def cycleMode(season,avgPrice): 
    price = (math.sin(2* season/24)/25 + avgPrice) + (math.sin(50*season*(random.randint(0, 100)/100))/100)
    return price                